### 업로드 폼

In [10]:
import sys
sys.path.insert(0, '../../assets/modules')
import pandas as pd
from callDB import DBHelper as db
from truckEDA import EDAHelper
from dotenv import dotenv_values
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime as dt
import os
import glob
import re
import pandas as pd
import datetime
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.dates as mdates
from dateutil import rrule
pd.set_option('display.max_rows', 300)
pd.options.display.float_format = '{:,.3f}'.format
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 필요시 run 전체 데이터프레임 보여주는 옵션
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

config = dotenv_values("../../assets/modules/.env")

server = "REL"
f = db(config, server)
eda = EDAHelper()

# 업로드 거래 중  CONVERT(CHAR(10), DATETIME, 23)
data =  f.table('SELECT d.DAL_NAME, d.DAL_COM_NM, d.DAL_GRADE, m.MCR_REG_DT, m.MCR_ID, m.MCR_SCD, m.MCR_BRAND_CD, m.MCR_MODEL_CD, m.MCR_LCD, m.MCR_PRICE, m.MCR_STATE, m.MCR_IMG1, m.MCR_IMG2,m.MCR_IMG3, m.MCR_IMG4, m.MCR_IMG5, m.MCR_IMG6, m.MCR_IMG7, m.MCR_IMG8, m.MCR_IMG9, m.MCR_IMG10, m.MCR_TON, m.MCR_YEAR_MM, m.MCR_KM, m.MCR_DESC \
FROM TB_MYCAR AS m \
LEFT OUTER JOIN  \
TB_DEALER AS d  \
ON m.DAL_ID = d.DAL_ID \
WHERE m.MCR_STATE = "C" ORDER BY FIELD(d.DAL_GRADE, "G", "D") DESC, d.DAL_GRADE ASC;')

def iTruck_data(data):
    based_data = data.astype('str')
    # data['MCR_REG_DT'] = data['MCR_REG_DT'].str.slice(0, 10)
    # based_data = data[data['MCR_REG_DT'].isin(data['CURRENT_DATE()'])]
    based_data = eda.rename_column(based_data, ['TB_MYCAR', 'TB_DEALER'])
    based_data['대분류'] = eda.rename_value(based_data['대분류'])
    based_data['소분류'] = eda.rename_value(based_data['소분류'])
    based_data['브랜드'] = eda.rename_value(based_data['브랜드'])
    based_data['모델'] = eda.rename_value(based_data['모델'])
    based_data['등록일자'] = pd.to_datetime(based_data['등록일자'], format='%Y-%m-%d', errors='coerce')

    based_data['상품명'] = based_data['브랜드'] + " " + based_data['모델'] + " " + based_data['소분류'] + " " + based_data['톤수'] + "톤"
    based_data["상품명"]= based_data["상품명"].str.replace("0톤", "", )
    based_data['추가이미지'] = based_data['이미지2'] + "\n" + based_data['이미지3'] + "\n" + based_data['이미지4'] + "\n" + based_data['이미지5'] + "\n" + based_data['이미지6'] + "\n" + based_data['이미지7'] + "\n" + based_data['이미지8'] + "\n" + based_data['이미지9']
    # based_data['추가이미지'] = based_data['추가이미지'].replace(np.nan, '') 
    based_data["추가이미지"]= based_data["추가이미지"].str.replace("None", "", )
    based_data['상세설명'] = "<h1><b>" + based_data['성명'] + "딜러</b></h1><h1><b>" + based_data['상사명'] + "</b></h1><br><h1><b>연식 : " + based_data['연식'] + "</b></h1><br><h1><b>톤수 : " + based_data['톤수'] + "</b></h1><br><h1><b>주행거리 : " + based_data['주행거리'] + "</b></h1><br><h1><b>" + based_data['상세설명'] + "</b></h1><br> <img src=\"https://i.ibb.co/y5JVWWw/stmart-store-img.jpg\" alt=\"stmart-store-img\" border=\"0\" width=\"800\" height=\"auto\">\n<img src=\"" + based_data['이미지1'] + "\" width=\"800\" height=\"auto\"> \n<img src=\"" + based_data['이미지2'] + "\" width=\"800\" height=\"auto\"> \n<img src=\"" + based_data['이미지3'] + "\" width=\"800\" height=\"auto\"> \n<img src=\"" + based_data['이미지4'] + "\" width=\"800\" height=\"auto\"> \n<img src=\"" + based_data['이미지5'] + "\" width=\"800\" height=\"auto\"> \n<img src=\"" + based_data['이미지6'] + "\" width=\"800\" height=\"auto\"> \n<img src=\"" + based_data['이미지7'] + "\" width=\"800\" height=\"auto\"> \n<img src=\"" + based_data['이미지8'] + "\" width=\"800\" height=\"auto\"> \n<img src=\"" + based_data['이미지9'] + "\" width=\"800\" height=\"auto\">\n<img src=\"https://i.ibb.co/y5JVWWw/stmart-store-img.jpg\" alt=\"stmart-store-img\" border=\"0\" width=\"800\" height=\"auto\">"
    based_data["상세설명"]= based_data["상세설명"].str.replace("<img src=\"None\" width=\"800\" height=\"auto\">", "", )  
    based_data["상세설명"]= based_data["상세설명"].str.replace("<img src=\"\" width=\"800\" height=\"auto\">", "", )
    based_data["상세설명"]= based_data["상세설명"].str.replace("직거래", "거래", )
    based_data.rename(columns = {'내차사기 아이디':'판매자 상품코드','브랜드':'제조사' ,'가격':'판매가', '이미지1':'대표이미지'},inplace=True)
    based_data = based_data.drop(['성명', '상사명', '이미지2', '이미지3', '이미지4', '이미지5', '이미지6', '이미지7', '이미지8', '이미지9', '이미지10', '소분류', '모델', '대분류', '매물상태', '톤수', '연식', '주행거리'], axis=1)
    return based_data

df3 = pd.read_excel('smartstore_test_1.xlsx', sheet_name="일괄등록", engine='openpyxl')
df3 = df3.rename(columns=df3.iloc[0])
df3 = df3.drop(index=[0,1,2,3,4,5], axis=0)

def upload(df3, based_data):
    df_12_axis0 = pd.concat([df3, based_data]) # row bind : axis = 0, default
    df_12_axis0["재고수량"] = df_12_axis0["재고수량"].replace(np.nan, 1)
    df_12_axis0["카테고리코드"] = df_12_axis0["카테고리코드"].replace(np.nan, '50004126')
    df_12_axis0["A/S 전화번호"] = df_12_axis0["A/S 전화번호"].replace(np.nan, '02-4949-8989')
    df_12_axis0["A/S 안내"] = df_12_axis0["A/S 안내"].replace(np.nan, '02-4949-8989')
    df_12_axis0["원산지코드"] = df_12_axis0["원산지코드"].replace(np.nan, '03')
    df_12_axis0["상품상태"] = df_12_axis0["상품상태"].replace(np.nan, '중고상품')
    df_12_axis0['판매가'] =  df_12_axis0['판매가'] + "0000"
    df_12_axis0 = df_12_axis0.replace(np.nan, '')
    return df_12_axis0

based_data = iTruck_data(data)
df_12_axis0 = upload(df3, based_data)
writer = pd.ExcelWriter('smartstore_upload2_test3.xlsx', engine='openpyxl')
pd.concat([
    df_12_axis0,
]).to_excel(writer, 'store', index=False)

writer.save()

C:\Users\joowo\AppData\Local\Temp\ipykernel_18976\3004368685.py:21: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
import sys
sys.path.insert(0, '../../assets/modules')
import pandas as pd
from callDB import DBHelper as db
from truckEDA import EDAHelper
from dotenv import dotenv_values
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime as dt
import os
import glob
import re
import pandas as pd
import datetime
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.dates as mdates
from dateutil import rrule
pd.set_option('display.max_rows', 300)
pd.options.display.float_format = '{:,.3f}'.format
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 필요시 run 전체 데이터프레임 보여주는 옵션
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

config = dotenv_values("../../assets/modules/.env")

server = "REL"
f = db(config, server)
eda = EDAHelper()

# 업로드 거래 중  CURRENT_DATE(),
data =  f.table('SELECT m.MCR_REG_DT, m.MCR_ID, m.MCR_SCD, m.MCR_BRAND_CD, m.MCR_MODEL_CD, m.MCR_LCD, m.MCR_PRICE, m.MCR_STATE, m.MCR_IMG1, m.MCR_IMG2,m.MCR_IMG3, m.MCR_IMG4, m.MCR_IMG5, m.MCR_IMG6, m.MCR_IMG7, m.MCR_IMG8, m.MCR_IMG9, m.MCR_IMG10, m.MCR_TON, m.MCR_YEAR_MM, m.MCR_KM, m.MCR_DESC \
FROM TB_MYCAR AS m \
LEFT OUTER JOIN  \
TB_DEALER AS d  \
ON m.DAL_ID = d.DAL_ID \
WHERE m.MCR_STATE = "C" \
ORDER BY m.MCR_REG_DT DESC LIMIT 100;')

based_data = data.astype('str')
data['MCR_REG_DT'] = data['MCR_REG_DT'].str.slice(0, 10)
based_data = data[data2['MCR_REG_DT'].isin(data['CURRENT_DATE()'])]
based_data = eda.rename_column(based_data, ['TB_MYCAR', 'TB_DEALER'])
based_data['대분류'] = eda.rename_value(based_data['대분류'])
based_data['소분류'] = eda.rename_value(based_data['소분류'])
based_data['브랜드'] = eda.rename_value(based_data['브랜드'])
based_data['모델'] = eda.rename_value(based_data['모델'])
based_data['등록일자'] = pd.to_datetime(based_data['등록일자'], format='%Y-%m-%d', errors='coerce')

based_data['상품명'] = based_data['브랜드'] + " " + based_data['모델'] + " " + based_data['소분류'] + " " + based_data['톤수'] + "톤"
based_data["상품명"]= based_data["상품명"].str.replace("0톤", "", )
based_data['추가이미지'] = based_data['이미지2'] + "\n" + based_data['이미지3'] + "\n" + based_data['이미지4'] + "\n" + based_data['이미지5'] + "\n" + based_data['이미지6'] + "\n" + based_data['이미지7'] + "\n" + based_data['이미지8'] + "\n" + based_data['이미지9']
# based_data['추가이미지'] = based_data['추가이미지'].replace(np.nan, '') 
based_data["추가이미지"]= based_data["추가이미지"].str.replace("None", "", )
based_data['상세설명'] = "연식 : " + based_data['연식'] + "<p>  톤수 : " + based_data['톤수'] + "<p>  주행거리 : " + based_data['주행거리'] + "<p> " + based_data['상세설명'] + "<p> \n<img src=\"" + based_data['이미지1'] + "\"> \n<img src=\"" + based_data['이미지2'] + "\"> \n<img src=\"" + based_data['이미지3'] + "\"> \n<img src=\"" + based_data['이미지4'] + "\"> \n<img src=\"" + based_data['이미지5'] + "\"> \n<img src=\"" + based_data['이미지6'] + "\"> \n<img src=\"" + based_data['이미지7'] + "\"> \n<img src=\"" + based_data['이미지8'] + "\"> \n<img src=\"" + based_data['이미지9'] + "\">"
based_data["상세설명"]= based_data["상세설명"].str.replace("<img src=\"None\">", "", )
based_data["상세설명"]= based_data["상세설명"].str.replace("직거래", "거래", )
based_data.rename(columns = {'내차사기 아이디':'판매자 상품코드','브랜드':'제조사' ,'가격':'판매가', '이미지1':'대표이미지'},inplace=True)
based_data = based_data.drop(['이미지2', '이미지3', '이미지4', '이미지5', '이미지6', '이미지7', '이미지8', '이미지9', '이미지10', '등록일자', '소분류', '모델', '대분류', '매물상태', '톤수', '연식', '주행거리'], axis=1)

df3 = pd.read_excel('smartstore_test_1.xlsx', sheet_name="일괄등록", engine='openpyxl')
df3 = df3.rename(columns=df3.iloc[0])
df3 = df3.drop(index=[0,1,2,3,4,5], axis=0)

df_12_axis0 = pd.concat([df3, based_data]) # row bind : axis = 0, default
df_12_axis0["재고수량"] = df_12_axis0["재고수량"].replace(np.nan, 1)
df_12_axis0["카테고리코드"] = df_12_axis0["카테고리코드"].replace(np.nan, '50004126')
df_12_axis0["A/S 전화번호"] = df_12_axis0["A/S 전화번호"].replace(np.nan, '02-4949-8989')
df_12_axis0["A/S 안내"] = df_12_axis0["A/S 안내"].replace(np.nan, '02-4949-8989')
df_12_axis0["원산지코드"] = df_12_axis0["원산지코드"].replace(np.nan, '03')
df_12_axis0["상품상태"] = df_12_axis0["상품상태"].replace(np.nan, '중고상품')
df_12_axis0['판매가'] =  df_12_axis0['판매가'] + "0000"
df_12_axis0 = df_12_axis0.replace(np.nan, '')

writer = pd.ExcelWriter('smartstore_upload2_test2.xlsx', engine='openpyxl')
pd.concat([
    df_12_axis0,
]).to_excel(writer, 'store', index=False)

writer.save()

## 수정 업로드 폼

In [ ]:
import sys
sys.path.insert(0, '../../assets/modules')
import pandas as pd
from callDB import DBHelper as db
from truckEDA import EDAHelper
from dotenv import dotenv_values
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime as dt
import os
import glob
import re
import pandas as pd
import datetime
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.dates as mdates
from dateutil import rrule
pd.set_option('display.max_rows', 300)
pd.options.display.float_format = '{:,.3f}'.format
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 필요시 run 전체 데이터프레임 보여주는 옵션
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

config = dotenv_values("../../assets/modules/.env")

server = "REL"
f = db(config, server)
eda = EDAHelper()

# 업로드 거래 중  CURRENT_DATE(),
data =  f.table('SELECT MCR_ID, MCR_DEAL_DT, MCR_REG_DT \
FROM TB_MYCAR  \
WHERE MCR_STATE != "H" \
ORDER BY MCR_REG_DT DESC;')

writer = pd.ExcelWriter('smartstore_modify_test.xlsx', engine='openpyxl')
pd.concat([
    data,
]).to_excel(writer, 'store', index=False)

writer.save()

In [ ]:
# 업로드 거래 중  CONVERT(CHAR(10), DATETIME, 23)
data =  f.table('SELECT CURRENT_DATE(), m.MCR_REG_DT, m.MCR_ID, m.MCR_SCD, m.MCR_BRAND_CD, m.MCR_MODEL_CD, m.MCR_LCD, m.MCR_PRICE, m.MCR_STATE, m.MCR_IMG1, m.MCR_IMG2,m.MCR_IMG3, m.MCR_IMG4, m.MCR_IMG5, m.MCR_IMG6, m.MCR_IMG7, m.MCR_IMG8, m.MCR_IMG9, m.MCR_IMG10, m.MCR_TON, m.MCR_YEAR_MM, m.MCR_KM, m.MCR_DESC \
FROM TB_MYCAR AS m \
LEFT OUTER JOIN  \
TB_DEALER AS d  \
ON m.DAL_ID = d.DAL_ID \
WHERE m.MCR_STATE = "C";')

data = data.astype('str')
data['MCR_REG_DT'] = data['MCR_REG_DT'].str.slice(0, 10)
based_data = data[data2['MCR_REG_DT'].isin(data['CURRENT_DATE()'])]
based_data = eda.rename_column(based_data, ['TB_MYCAR', 'TB_DEALER'])
based_data['대분류'] = eda.rename_value(based_data['대분류'])
based_data['소분류'] = eda.rename_value(based_data['소분류'])
based_data['브랜드'] = eda.rename_value(based_data['브랜드'])
based_data['모델'] = eda.rename_value(based_data['모델'])
based_data['등록일자'] = pd.to_datetime(based_data['등록일자'], format='%Y-%m-%d', errors='coerce')

based_data['상품명'] = based_data['브랜드'] + " " + based_data['모델'] + " " + based_data['소분류']
based_data['추가이미지'] = based_data['이미지2'] + "\n" + based_data['이미지3'] + "\n" + based_data['이미지4'] + "\n" + based_data['이미지5'] + "\n" + based_data['이미지6'] + "\n" + based_data['이미지7'] + "\n" + based_data['이미지8'] + "\n" + based_data['이미지9']
# based_data['추가이미지'] = based_data['추가이미지'].replace(np.nan, '') 
based_data["추가이미지"]= based_data["추가이미지"].str.replace("None", "", )
based_data['상세설명'] = "연식 : " + based_data['연식'] + "<p>  톤수 : " + based_data['톤수'] + "<p>  주행거리 : " + based_data['주행거리'] + "<p> " + based_data['상세설명'] + "<p> \n<img src=\"" + based_data['이미지1'] + "\"> \n<img src=\"" + based_data['이미지2'] + "\"> \n<img src=\"" + based_data['이미지3'] + "\"> \n<img src=\"" + based_data['이미지4'] + "\"> \n<img src=\"" + based_data['이미지5'] + "\"> \n<img src=\"" + based_data['이미지6'] + "\"> \n<img src=\"" + based_data['이미지7'] + "\"> \n<img src=\"" + based_data['이미지8'] + "\"> \n<img src=\"" + based_data['이미지9'] + "\">"
based_data["상세설명"]= based_data["상세설명"].str.replace("<img src=\"None\">", "", )
based_data["상세설명"]= based_data["상세설명"].str.replace("직거래", "거래", )
based_data.rename(columns = {'내차사기 아이디':'판매자 상품코드','브랜드':'제조사' ,'가격':'판매가', '이미지1':'대표이미지'},inplace=True)
based_data = based_data.drop(['이미지2', '이미지3', '이미지4', '이미지5', '이미지6', '이미지7', '이미지8', '이미지9', '이미지10','CURRENT_DATE()', '등록일자', '소분류', '모델', '대분류', '매물상태', '톤수', '연식', '주행거리'], axis=1)

df3 = pd.read_excel('smartstore_upload.xlsx', sheet_name="일괄등록", engine='openpyxl')
df3 = df3.rename(columns=df3.iloc[0])
df3 = df3.drop(index=[0,1,2,3,4,5], axis=0)

df_12_axis0 = pd.concat([df3, based_data]) # row bind : axis = 0, default
df_12_axis0["재고수량"] = df_12_axis0["재고수량"].replace(np.nan, 1)
df_12_axis0["카테고리코드"] = df_12_axis0["카테고리코드"].replace(np.nan, '50004126')
df_12_axis0["A/S 전화번호"] = df_12_axis0["A/S 전화번호"].replace(np.nan, '02-4949-8989')
df_12_axis0["A/S 안내"] = df_12_axis0["A/S 안내"].replace(np.nan, '02-4949-8989')
df_12_axis0["원산지코드"] = df_12_axis0["원산지코드"].replace(np.nan, '03')
df_12_axis0["상품상태"] = df_12_axis0["상품상태"].replace(np.nan, '중고상품')
df_12_axis0['판매가'] =  df_12_axis0['판매가'] + "0000"
df_12_axis0 = df_12_axis0.replace(np.nan, '')

writer = pd.ExcelWriter('smartstore_upload2_test.xlsx', engine='openpyxl')
pd.concat([
    df_12_axis0,
]).to_excel(writer, 'store', index=False)

writer.save()